In [1]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
import ipdb
import re
from collections import Counter

import matplotlib.pyplot as plt
# import mplcursors
import seaborn as sns
%matplotlib inline
sns.set(style='darkgrid', context='notebook', rc={'figure.figsize':(14,10)}, font_scale=2)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('chained_assignment',None)

# Set random seeds for reproducibility on a specific machine
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)

RandomState(MT19937) at 0x13FD19E40

In [2]:
df = pd.read_csv('../../data/cong_data.tsv', sep='\t')

In [3]:
group = df['group'].values
group

array([1, 1, 1, ..., 0, 1, 0])

In [8]:
Counter(df['Feeling'].values)

Counter({'warm': 2185, 'cold': 328, 'neutral': 310, 'dunno': 209, 'mixed': 1})

In [7]:
df['affect'] = df.apply(lambda x: 1 if (x['Feeling']=='warm' or x['Behavior']=='app') else 0, axis=1)

In [8]:
affect = df['affect'].values
affect

array([1, 1, 1, ..., 1, 1, 1])

In [28]:
Counter(affect)

Counter({1: 2281, 0: 752})

In [29]:
np.round(Counter(affect)[1]/(Counter(affect)[1]+Counter(affect)[0]), 3)

0.752

In [10]:
Counter(group)

Counter({1: 1516, 0: 1517})

In [13]:
spec = df['Specificity'].values
spec

array([3.761914, 4.046838, 3.608603, ..., 4.258683, 4.266755, 4.178448])

In [14]:
from scipy.stats import spearmanr, pearsonr

In [15]:
pearsonr(group, affect)

(0.22734386366789328, 7.470915836795385e-37)

In [16]:
spearmanr(group, affect)

SpearmanrResult(correlation=0.227343863667894, pvalue=7.470915836818142e-37)

In [17]:
pearsonr(group, spec)

(-0.07295593127914998, 5.780853913771975e-05)

In [18]:
spearmanr(group, spec)

SpearmanrResult(correlation=-0.0680496087043887, pvalue=0.00017649827101029938)

In [26]:
df.sample(5)

,TweetId,tweet,username,mentname,Date,Predom,Feeling,Behavior,Admiration,Admiration_Int,Anger,Anger_Int,Disgust,Disgust_Int,Fear,Fear_Int,Interest,Interest_Int,Joy,Joy_Int,Sadness,Sadness_Int,Surprise,Surprise_Int,group,party,Split,Specificity,affect
1924,629389022222127105,"Thx @repnewhouse for supporting #SIGMA, my bill to create an #Obamacare Special Inspector General →",peterroskam,repnewhouse,2015-08-06,predom,warm,app,False,2.000000,False,2.0,False,2.0,False,2.0,False,2.000000,True,2.333333,False,2.0,False,2.0,1,R,train,3.644890,1
73,1031634800967266305,"Thanks, @repblumenauer for joining me in demanding #PaperBallotsNOW. #PAVEAct",ronwyden,repblumenauer,2018-08-20,predom,warm,app,False,2.000000,False,2.0,False,2.0,False,2.0,False,1.666667,True,2.000000,False,2.0,False,2.0,1,D,test,2.908854,1
65,1026468183652196353,"I stood with @repbrianfitz, widow Ellen Saracini, and North Jersey police officers, firefighters, local elected officials, pilots, and flight attendants to call on Congress to pass HR 911, the Saracini Aviation Safety Act of 2017.",repjoshg,repbrianfitz,2018-08-06,predom,warm,app,False,2.000000,False,2.0,False,2.0,False,2.0,True,2.000000,False,2.000000,False,2.0,False,2.0,0,D,test,4.442910,1
1490,460891102443487232,Appreciate Chairman @darrellissa's leadership in bringing this important legislation to the floor:,repscottrigell,darrellissa,2014-04-28,predom,warm,app,True,2.333333,False,2.0,False,2.0,False,2.0,False,2.000000,True,2.333333,False,2.0,False,2.0,1,R,train,3.042706,1
2441,821039790833410048,"⚡️ How @repjohnlewis, President Obama, other officials & I are honoring Dr. King. #MLKDAY",senmarkey,repjohnlewis,2017-01-16,predom,warm,app,False,2.000000,False,2.0,False,2.0,False,2.0,False,2.000000,False,2.000000,False,2.0,False,2.0,1,D,train,3.278839,1


In [32]:
np.round(np.mean(df['Specificity']),2)

3.49

In [33]:
np.round(np.std(df['Specificity']), 2)

0.53

In [37]:
newdf = df[(df['Specificity']<3)|(df['Specificity']>4)]

In [38]:
spearmanr(newdf['group'], newdf['Specificity'])

SpearmanrResult(correlation=-0.13345377946081396, pvalue=1.2802883983753608e-05)

In [39]:
pearsonr(newdf['group'], newdf['Specificity'])

(-0.13189700573478147, 1.617153340276441e-05)

In [49]:
df[(df['Split']=='train') & (df['affect']==0)].shape

(589, 29)

In [53]:
df.loc[:, ['Split', 'TweetId', 'username', 'mentname', 'Date', 'Feeling', 'Behavior', 'group', 'party', 'Specificity', 'affect']].to_csv('~/Downloads/data.tsv', sep='\t', index=False)